In [51]:
import os
from dotenv import load_dotenv
load_dotenv()

import json

from get_articles import get_data
from get_articles import process_articles

from importlib import reload
reload(get_data)
reload(process_articles)

import pprint
pp = pprint.PrettyPrinter(indent=2)

In [42]:
home = os.getenv("PROJ_HOME")

institutional_data_path = os.path.join(home, "data/institutional")
metadata_path = os.path.join(home, "data/subject_matter/metadata")
article_ids_path = os.path.join(home, "data/cache/article_ids.json")
data_out_dir = os.path.join(home, "data/subject_matter/encyclopedia")

In [368]:
# get encyclopedia metadata

def get_metadata(sources=["advanced"], path=None):
    articles = {}

    for source in sources:
        articles_in_source = get_data.get_encyclopedia_metadata(source, path)
        articles[source] = articles_in_source

    # remove articles with empty title fields
    for source in sources:
        for article in articles[source]:
            if article['title'] == '':
                articles[source].remove(article)

    if not os.path.exists(path):
        os.makedirs(path)

    for source in sources:
        filepath = os.path.join(path, source + '.json')

        if os.path.exists(filepath):
            os.remove(filepath)

        with open(filepath, "w") as outfile:
            outfile.write(json.dumps(articles[source], indent=2))

    return articles

In [369]:
advanced_metadata = get_metadata(path=metadata_path)

In [43]:
# load article_ids to get

with open(article_ids_path, "r") as infile:
    article_ids = json.load(infile)

In [58]:
# test the get_articles function
first_5_articles = get_data.get_articles(article_ids[:5])
df = process_articles.clean_articles(first_5_articles)
df

,Title,Alt Title,Field,Is Person,Text
62898,reciprocity,[],[],0,"reciprocity , in international trade, the gr..."
38514,Guna,[],[],0,"Guna , also spelled Goona , city, northern M..."
62859,Reading,[],[],0,"Reading: Eng., on the River Kennet © Amanda ..."
62860,Reading,[],[],0,"Reading: Pa. Reading , city, seat (1752) of ..."
3431,Abū Qīr Bay,[],[],0,"Abū Qīr Bay , also called Abukir Bay , or Ab..."


In [59]:
filepath = os.path.join(home, "data/subject_matter/encyclopedia/data.csv")
df.to_csv(filepath)